In [ ]:
! pip install -U langchain_groq langchain langchain_community langchain_openai tavily-python tiktoken langchainhub chromadb

# Tool calling agent with LLaMA3

[Tool calling](https://python.langchain.com/docs/modules/agents/agent_types/tool_calling/) allows an LLM to detect when one or more tools should be called.

It will then respond with the inputs that should be passed to those tools. 

LangChain has a general agent that works with tool-calling LLMs.

### Tools 

Let's define a few tools.

`Retriever`

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = Chroma.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

`Web search`

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

`Custom`

In [6]:
from langchain.agents import tool

@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2

In [8]:
tools = [magic_function, search, retriever_tool] 

### LLM

Here, we need a llama model that support tool use.

This can be accomplished via prompt engineering (e.g., see [here](https://replicate.com/hamelsmu/llama-3-70b-instruct-awq-with-tools)) or fine-tuning (e.g., see [here](https://huggingface.co/mzbac/llama-3-8B-Instruct-function-calling) and [here](https://huggingface.co/mzbac/llama-3-8B-Instruct-function-calling)).

We can review LLMs that support tool calling [here](https://python.langchain.com/docs/integrations/chat/) and Groq is included.

[Here](https://github.com/groq/groq-api-cookbook/blob/main/llama3-stock-market-function-calling/llama3-stock-market-function-calling.ipynb) is a reference for Groq + tool use.

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

### Agent

We use LangChain [tool calling agent](https://python.langchain.com/docs/modules/agents/agent_types/tool_calling/). 

In [11]:
# Prompt 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [12]:
### Run
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
agent_executor.invoke({"input": "what is the value of magic_function(3)?"})



> Entering new AgentExecutor chain...

Invoking: `magic_function` with `{'input': 3}`


5The result of `magic_function(3)` is indeed 5.

> Finished chain.


{'input': 'what is the value of magic_function(3)?',
 'output': 'The result of `magic_function(3)` is indeed 5.'}

Trace: 

https://smith.langchain.com/public/adf06494-94d6-4e93-98f3-60e65d2f2c19/r

In [14]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current weather in san francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714766520, 'localtime': '2024-05-03 13:02'}, 'current': {'last_updated_epoch': 1714766400, 'last_updated': '2024-05-03 13:00', 'temp_c': 17.8, 'temp_f': 64.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 250, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 54, 'cloud': 25, 'feelslike_c': 17.8, 'feelslike_f': 64.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 17.0, 'gust_kph': 27.4}}"}, {'url': 'https://www.wunderground.

{'input': 'whats the weather in sf?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 64°F (17.8°C) and humidity of 54%. The wind is blowing at 6.9 mph (11.2 km/h) from the west-southwest direction.'}

Trace: 

https://smith.langchain.com/public/64a62781-7e3c-4acf-ae72-ce49ccb82960/r

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

This last question failed to run. 

Trace:

https://smith.langchain.com/public/960a40e9-24f1-42a0-859d-2e0a30018d1c/r

We can see that the agent correctly decides to query the vectorstore for a question about LangSmith.

But it then inexplicably attempts web search. 

And it appears to get stuck in a loop of calling various tools before crashing.

Of course, this is using a non-fine-tuned (only prompting) version of llama3 for tool-use.

But, it illustates the reliability challenge with using Agent Executor. 

It is sensitive to the LLMs capacity for tool-use! 

In the next notebook, we will show an alternative way to implement this agent using LangGraph.